# Load Data

In [ ]:
import pandas as pd

file_path = 'PHIR82FL.DTA' #adjust the path when you run on your own laptop
df = pd.read_stata(file_path, convert_categoricals= False)

df

,caseid,v000,v001,v002,v003,v004,v005,v006,v007,v008,...,s615d_3,s615d_4,s615d_5,s615d_6,s617b_1,s617b_2,s617b_3,s617b_4,s617b_5,s617b_6
0,1 4 2,PH8,1,4,2,1,116381,5,2022,1469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 4 3,PH8,1,4,3,1,116381,5,2022,1469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1 4 4,PH8,1,4,4,1,116381,5,2022,1469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1 6 2,PH8,1,6,2,1,116381,5,2022,1469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1 7 6,PH8,1,7,6,1,116381,5,2022,1469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27816,1247 23 2,PH8,1247,23,2,1247,694550,5,2022,1469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27817,1247 23 4,PH8,1247,23,4,1247,694550,5,2022,1469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27818,1247 26 3,PH8,1247,26,3,1247,694550,5,2022,1469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27819,1247 26 7,PH8,1247,26,7,1247,694550,5,2022,1469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# EDA (Exploratory Data Analysis)

## Survey Answer Meaning

                               0  Never
                               1  Often
                               2  Sometimes
                               3  Yes, but not in the last 12 months
                               4  Yes, but frequency in last 12 months missing
                           (m) 9  Missing
                          (na)    Not applicable

Logic:
1. Technically, the meanings of 'missing' will be same as the 'na' value, here we assume they are together (tbh there is no value '9' in all the attribute columns)
2. Here, we are still using the 'smaller means more serious'. The computational method will be: SUM(all attribute values) / # not na or 9 or 0 col = avg of the valid answer
3. This algo should apply for each type of violence and the overall one
??: the formula should consider the number of vaild answer

In [ ]:
#the current data w/o the all na row
def check_column_distribution(dataframe):
    """
    Prints the distribution of values for each column in the dataframe.

    Parameters:
    dataframe (pd.DataFrame): The dataframe to check.
    """
    for col in dataframe.columns:
        try:
            print(f"Distribution for column: {col}")
            print(dataframe[col].value_counts(dropna=False))
        except ValueError as e:
            print(f"Could not process column '{col}' due to: {str(e)}")
        print("-" * 50)

In [ ]:
check_column_distribution(df)

Distribution for column: caseid
caseid
   1   4  2    1
 809  24  3    1
 810  16  8    1
 810  15  2    1
 810   9  2    1
              ..
 399   5  2    1
 399   4  7    1
 399   4  3    1
 399   4  1    1
1247  27  2    1
Name: count, Length: 27821, dtype: int64
--------------------------------------------------
Distribution for column: v000
v000
PH8    27821
Name: count, dtype: int64
--------------------------------------------------
Distribution for column: v001
v001
25      72
62      71
1095    64
29      58
28      53
        ..
395      4
1080     3
346      2
304      2
310      2
Name: count, Length: 1247, dtype: int64
--------------------------------------------------
Distribution for column: v002
v002
17      1052
16      1047
7       1039
10      1011
5       1010
        ... 
2022       1
1034       1
2001       1
3015       1
2009       1
Name: count, Length: 119, dtype: int64
--------------------------------------------------
Distribution for column: v003
v003
2     1

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = 'drive/MyDrive/DS Proj (Team 25)/Individual_csv/Bangladesh Standard DHS 2007.csv'

# Data Cleaning

In [ ]:
emotional_list = ['d101a', 'd101b', 'd101c', 'd101d', 'd101e', 'd101f', 'd103a', 'd103b']
physical_list = ['d105a', 'd105b', 'd105c', 'd105d', 'd105e', 'd105f', 'd105g']
sexual_list = ['d105h', 'd105i', 'd105k']

In [ ]:
attribute_lists = {
    'emotional': emotional_list,
    'physical': physical_list,
    'sexual': sexual_list
}

In [ ]:
#new
def data_cleaning(raw_data_path, attributes_list_path, attribute_lists):
    # Read the raw CSV dataset
    whole_data = pd.read_csv(raw_data_path)
    attributes_list = pd.read_csv(attributes_list_path)

    # Convert column names to lowercase
    whole_data.columns = whole_data.columns.str.lower()
    attributes_list['Column ID'] = attributes_list['Column ID'].str.lower()

    # Extract the list of column IDs
    column_ids = attributes_list['Column ID'].tolist()

    # Locate which columns from column_ids exist in the main dataset
    existing_columns = [col for col in column_ids if col in whole_data.columns]
    missing_columns = [col for col in column_ids if col not in whole_data.columns]

    # Select only the existing columns from the main dataset
    filtered_dataset = whole_data[existing_columns]

    # Drop the NA rows from the main dataset
    cleaned_data = filtered_dataset.dropna(how='all')

    # Separate rows where all values in the attribute lists are NA
    attribute_columns = [col for cols in attribute_lists.values() for col in cols if col in cleaned_data.columns]
    attribute_data = cleaned_data[attribute_columns]
    all_na_rows = cleaned_data[attribute_data.isna().all(axis=1)]
    cleaned_violence_data = cleaned_data[~cleaned_data.index.isin(all_na_rows.index)]

    # Reset the index so that it can be tracked by ID later
    cleaned_violence_data = cleaned_violence_data.reset_index(drop=True)
    cleaned_violence_data['unique_id'] = cleaned_violence_data.index + 1

    # Reorder columns to have 'unique_id' first
    cols = ['unique_id'] + [col for col in cleaned_violence_data.columns if col != 'unique_id']
    cleaned_violence_data = cleaned_violence_data[cols]

    return cleaned_violence_data, all_na_rows


In [ ]:
raw_data_path = 'PHIR82FL.csv'
attributes_list_path = 'Attributes_Template.csv'
#clean_data = data_cleaning(raw_data_path, attributes_list_path)
clean_data, all_na_rows = data_cleaning(raw_data_path, attributes_list_path, attribute_lists)

print(clean_data)

C:\Users\wangc\AppData\Local\Temp\ipykernel_34812\2008153363.py:4: DtypeWarning: Columns (4722) have mixed types. Specify dtype option on import or set low_memory=False.
  whole_data = pd.read_csv(raw_data_path)


       unique_id v000  v001  v002  v003  v004    v005  d103e  d105h  d105h  \
0              1  PH8     1     6     2     1  116381    0.0    0.0    0.0   
1              2  PH8     1     7     6     1  116381    0.0    0.0    0.0   
2              3  PH8     1     8     2     1  116381    0.0    0.0    0.0   
3              4  PH8     1     9     2     1  116381    0.0    0.0    0.0   
4              5  PH8     1    17     2     1  116381    0.0    2.0    2.0   
...          ...  ...   ...   ...   ...   ...     ...    ...    ...    ...   
14581      14582  PH8  1247    17     3  1247  694550    0.0    0.0    0.0   
14582      14583  PH8  1247    20     2  1247  694550    0.0    0.0    0.0   
14583      14584  PH8  1247    23     2  1247  694550    0.0    0.0    0.0   
14584      14585  PH8  1247    26     7  1247  694550    0.0    0.0    0.0   
14585      14586  PH8  1247    27     2  1247  694550    0.0    0.0    0.0   

       ...  d105e  d105f  d105g  d105h  d105h  d105i  d105i  d1

In [ ]:
print(all_na_rows)

      v000  v001  v002  v003  v004    v005  d103e  d105h  d105i  d105k  ...  \
0      PH8     1     4     2     1  116381    NaN    NaN    NaN    NaN  ...   
1      PH8     1     4     3     1  116381    NaN    NaN    NaN    NaN  ...   
2      PH8     1     4     4     1  116381    NaN    NaN    NaN    NaN  ...   
7      PH8     1    14     7     1  116381    NaN    NaN    NaN    NaN  ...   
13     PH8     1    27     2     1  116381    NaN    NaN    NaN    NaN  ...   
...    ...   ...   ...   ...   ...     ...    ...    ...    ...    ...  ...   
27809  PH8  1247     3     3  1247  694550    NaN    NaN    NaN    NaN  ...   
27810  PH8  1247     6     2  1247  694550    NaN    NaN    NaN    NaN  ...   
27811  PH8  1247     6     7  1247  694550    NaN    NaN    NaN    NaN  ...   
27817  PH8  1247    23     4  1247  694550    NaN    NaN    NaN    NaN  ...   
27818  PH8  1247    26     3  1247  694550    NaN    NaN    NaN    NaN  ...   

       d105b  d105c  d105d  d105e  d105f  d105g  d1

# requalify

In [ ]:
y_list=['d101a', 'd101b', 'd101c', 'd101d', 'd101e', 'd101f', 'd103a',
            'd103b','d105a', 'd105b', 'd105c', 'd105d', 'd105e', 'd105f', 'd105g',
            'd105h', 'd105i', 'd105k']

In [ ]:
# drop the duplicated row
clean = clean_data.loc[:, ~clean_data.columns.duplicated()]

In [ ]:
clean[y_list]=clean[y_list].replace({0:1,1:4,2:3,3:2})

C:\Users\wangc\AppData\Local\Temp\ipykernel_34812\3600627075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean[y_list]=clean[y_list].replace({0:1,1:4,2:3,3:2})


In [ ]:
clean_data=clean
#62 cloumns

In [ ]:
'''y_list=['d101a', 'd101b', 'd101c', 'd101d', 'd101e', 'd101f', 'd103a',
            'd103b','d105a', 'd105b', 'd105c', 'd105d', 'd105e', 'd105f', 'd105g',
            'd105h', 'd105i', 'd105k']
clean_data['d105d'].value_counts()

#d101f d105g  all nan'''

d105d
0.0    14303
3.0      142
2.0      128
1.0       13
Name: count, dtype: int64

# Response Variable (y) Calculation

In [ ]:
def compute_valid_and_sum(df, attribute_lists):
    # Initialize overall metrics
    df['overall_valid_count'] = 0
    df['overall_valid_sum'] = 0.0

    for name, columns in attribute_lists.items():
        existing_columns = [col for col in columns if col in df.columns]
        if not existing_columns:  # If no columns exist, set metrics to 0
            df[f'{name}_valid_count'] = 0
            df[f'{name}_valid_sum'] = 0.0
            df[f'{name}_avg'] = 0
        else:
            # Replace 0 and 9 with NaN to consider them as invalid
            valid_df = df[existing_columns].replace([0, 9], float('nan'))
            # Compute valid counts and sum for the attribute list
            df[f'{name}_valid_count'] = valid_df.notna().sum(axis=1)
            df[f'{name}_valid_sum'] = valid_df.sum(axis=1)
            # Calculate average, handling division by zero by filling NaN with 0
            df[f'{name}_avg'] = (df[f'{name}_valid_sum'] / df[f'{name}_valid_count']).fillna(0)

        # Update the overall valid count and sum
        df['overall_valid_count'] += df[f'{name}_valid_count']
        df['overall_valid_sum'] += df[f'{name}_valid_sum']

    # Compute the overall average and handle division by zero
    df['overall_avg'] = (df['overall_valid_sum'] / df['overall_valid_count']).fillna(0)

    return df


In [ ]:
result = compute_valid_and_sum(clean_data, attribute_lists)
print(result)

       unique_id v000  v001  v002  v003  v004    v005  d103e  d105h  d105i  \
0              1  PH8     1     6     2     1  116381    0.0    1.0    1.0   
1              2  PH8     1     7     6     1  116381    0.0    1.0    1.0   
2              3  PH8     1     8     2     1  116381    0.0    1.0    1.0   
3              4  PH8     1     9     2     1  116381    0.0    1.0    1.0   
4              5  PH8     1    17     2     1  116381    0.0    3.0    3.0   
...          ...  ...   ...   ...   ...   ...     ...    ...    ...    ...   
14581      14582  PH8  1247    17     3  1247  694550    0.0    1.0    1.0   
14582      14583  PH8  1247    20     2  1247  694550    0.0    1.0    1.0   
14583      14584  PH8  1247    23     2  1247  694550    0.0    1.0    1.0   
14584      14585  PH8  1247    26     7  1247  694550    0.0    1.0    1.0   
14585      14586  PH8  1247    27     2  1247  694550    0.0    1.0    1.0   

       ...  emotional_valid_count  emotional_valid_sum  emotion

C:\Users\wangc\AppData\Local\Temp\ipykernel_34812\1687615479.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overall_valid_count'] = 0
C:\Users\wangc\AppData\Local\Temp\ipykernel_34812\1687615479.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overall_valid_sum'] = 0.0
C:\Users\wangc\AppData\Local\Temp\ipykernel_34812\1687615479.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [ ]:
result.to_csv('final.csv', index=False)

# Metadata Mapping

In [ ]:
import pandas as pd

df=pd.read_csv('final.csv')

df

,unique_id,v000,v001,v002,v003,v004,v005,d103e,d105h,d105i,...,emotional_valid_count,emotional_valid_sum,emotional_avg,physical_valid_count,physical_valid_sum,physical_avg,sexual_valid_count,sexual_valid_sum,sexual_avg,overall_avg
0,1,PH8,1,6,2,1,116381,0.0,1.0,1.0,...,7,7.0,1.000000,6,6.0,1.0,3,3.0,1.0,1.0000
1,2,PH8,1,7,6,1,116381,0.0,1.0,1.0,...,7,13.0,1.857143,6,6.0,1.0,3,3.0,1.0,1.3750
2,3,PH8,1,8,2,1,116381,0.0,1.0,1.0,...,7,7.0,1.000000,6,6.0,1.0,3,3.0,1.0,1.0000
3,4,PH8,1,9,2,1,116381,0.0,1.0,1.0,...,7,7.0,1.000000,6,6.0,1.0,3,3.0,1.0,1.0000
4,5,PH8,1,17,2,1,116381,0.0,3.0,3.0,...,7,11.0,1.571429,6,6.0,1.0,3,9.0,3.0,1.6250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14581,14582,PH8,1247,17,3,1247,694550,0.0,1.0,1.0,...,7,7.0,1.000000,6,6.0,1.0,3,3.0,1.0,1.0000
14582,14583,PH8,1247,20,2,1247,694550,0.0,1.0,1.0,...,7,9.0,1.285714,6,6.0,1.0,3,3.0,1.0,1.1250
14583,14584,PH8,1247,23,2,1247,694550,0.0,1.0,1.0,...,7,8.0,1.142857,6,6.0,1.0,3,3.0,1.0,1.0625
14584,14585,PH8,1247,26,7,1247,694550,0.0,1.0,1.0,...,7,7.0,1.000000,6,6.0,1.0,3,3.0,1.0,1.0000


In [ ]:
y_list=['d101a', 'd101b', 'd101c', 'd101d', 'd101e', 'd101f', 'd103a',
            'd103b','d105a', 'd105b', 'd105c', 'd105d', 'd105e', 'd105f', 'd105g',
            'd105h', 'd105i', 'd105k']

emotional_list = ['d101a', 'd101b', 'd101c', 'd101d', 'd101e', 'd101f', 'd103a', 'd103b']
physical_list = ['d105a', 'd105b', 'd105c', 'd105d', 'd105e', 'd105f', 'd105g']
sexual_list = ['d105h', 'd105i', 'd105k']


attribute_lists = {
    'emotional': emotional_list,
    'physical': physical_list,
    'sexual': sexual_list
}

In [ ]:
import pandas as pd

def metadata(metadata_path):
    try:
        metadata = pd.read_csv(metadata_path, encoding='utf-8')
    except UnicodeDecodeError:
        # Try a different encoding if UTF-8 does not work
        metadata = pd.read_csv(metadata_path, encoding='ISO-8859-1')
    metadata['Item Name'] = metadata['Item Name'].apply(lambda x: x.lower() if isinstance(x, str) else x)
    return metadata

metadata_path = 'variable_description.csv'
metadata = metadata(metadata_path)
metadata


,Item Name,Item Label,Code,Description
0,caseid,(id) Case Identification,NaN,record type
1,v000,Country code and phase,NaN,NaN
2,v001,Cluster number,NaN,NaN
3,v002,Household number,NaN,NaN
4,v003,Respondent's line number,NaN,NaN
...,...,...,...,...
7358,s614a,Days after diarrhea begun sought advice or trea,0.065972222,Number of days
7359,s615d,Child given probiotic,0,No
7360,s615d,Child given probiotic,1,Yes
7361,s615d,Child given probiotic,8,Don't know


In [ ]:
new_des=['Never','Yes, but not in the last 12 months','Sometimes','Often']


for i in y_list:
    indices = metadata[metadata['Item Name'] == i].index
    for num in range(4):
        metadata.loc[indices[num+1], 'Description'] = new_des[num]

In [ ]:
# Assuming your reference data is structured as given, you might need to adjust keys if they differ
# Create a dictionary for each column that you need to map
mappings = {}
for item in metadata['Item Name'].unique():
    mappings[item] = dict(zip(metadata[metadata['Item Name'] == item]['Code'],
                              metadata[metadata['Item Name'] == item]['Description']))


In [ ]:
mappings

{'caseid': {nan: 'record type'},
 'v000': {nan: nan},
 'v001': {nan: nan},
 'v002': {nan: nan},
 'v003': {nan: nan},
 'v004': {nan: nan},
 'v005': {nan: nan},
 'v006': {'1:12': nan},
 'v007': {'2022': nan},
 'v008': {nan: nan},
 'v008a': {nan: nan},
 'v009': {'1:12': nan, 'na': 'Not applicable'},
 'v010': {'1972:2007': nan, 'na': 'Not applicable'},
 'v011': {nan: nan},
 'v012': {'15:49': nan, 'na': 'Not applicable'},
 'v013': {'1': '15-19',
  '2': '20-24',
  '3': '25-29',
  '4': '30-34',
  '5': '35-39',
  '6': '40-44',
  '7': '45-49',
  'na': 'Not applicable'},
 'v014': {'1': 'Month and year - information complete',
  '2': 'Month and age - year imputed',
  '3': 'Year and age - month imputed',
  '4': 'Year and age - year ignored',
  '5': 'Year - age/month imputed',
  '6': 'Age - year/month imputed',
  '7': 'Month - age/year imputed',
  '8': 'None - all imputed',
  'na': 'Not applicable'},
 'v015': {'1': 'Completed',
  '2': 'Not at home',
  '3': 'Postponed',
  '4': 'Refused',
  '5': 'Par

In [ ]:
def apply_mappings(data, mappings):
    """
    Apply mappings to specified columns of a DataFrame.

    Parameters:
    - data (pd.DataFrame): The DataFrame to be modified.
    - mappings (dict): A dictionary where keys are column names and values are dictionaries
                       mapping old values to new values.

    Returns:
    - pd.DataFrame: The modified DataFrame with mapped values.
    """
    for column, mapping in mappings.items():
        if column in data.columns:
            # Convert mapping keys from string to appropriate type, if possible
            try:

                mapping_int_keys = {int(k): v for k, v in mapping.items()}

            except ValueError:
                # If conversion fails, use the mapping as is
                mapping_int_keys = mapping

            # Apply the mapping to the DataFrame column using map on the specific series
            data[column] = data[column].map(mapping_int_keys).fillna(data[column])

    return data


map_data_table = apply_mappings(df, mappings)


C:\Users\wangc\AppData\Local\Temp\ipykernel_34444\164241769.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[column] = data[column].map(mapping_int_keys).fillna(data[column])
C:\Users\wangc\AppData\Local\Temp\ipykernel_34444\164241769.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[column] = data[column].map(mapping_int_keys).fillna(data[column])
C:\Users\wangc\AppData\Local\Temp\ipykernel_34444\164241769.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. 

In [ ]:
for i in df.columns:
    try:
        name=list(metadata[metadata['Item Name']==i]['Item Label'])[0]
        df = df.rename(columns={i: name})
    except IndexError as e:
        print(f"no need to rename")

no need to rename
no need to rename
no need to rename
no need to rename
no need to rename
no need to rename
no need to rename
no need to rename
no need to rename
no need to rename
no need to rename
no need to rename
no need to rename


In [ ]:
df

,unique_id,Country code and phase,Cluster number,Household number,Respondent's line number,Ultimate area unit,Women's individual sample weight (6 decimals),Ever control your own money or properties or fo,Ever been physically forced into unwanted sex b,Ever been forced into other unwanted sexual act,...,emotional_valid_count,emotional_valid_sum,emotional_avg,physical_valid_count,physical_valid_sum,physical_avg,sexual_valid_count,sexual_valid_sum,sexual_avg,overall_avg
0,1,PH8,1.0,6.0,2.0,1.0,116381.0,Never,Never,Never,...,7,7.0,1.000000,6,6.0,1.0,3,3.0,1.0,1.0000
1,2,PH8,1.0,7.0,6.0,1.0,116381.0,Never,Never,Never,...,7,13.0,1.857143,6,6.0,1.0,3,3.0,1.0,1.3750
2,3,PH8,1.0,8.0,2.0,1.0,116381.0,Never,Never,Never,...,7,7.0,1.000000,6,6.0,1.0,3,3.0,1.0,1.0000
3,4,PH8,1.0,9.0,2.0,1.0,116381.0,Never,Never,Never,...,7,7.0,1.000000,6,6.0,1.0,3,3.0,1.0,1.0000
4,5,PH8,1.0,17.0,2.0,1.0,116381.0,Never,Sometimes,Sometimes,...,7,11.0,1.571429,6,6.0,1.0,3,9.0,3.0,1.6250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14581,14582,PH8,1247.0,17.0,3.0,1247.0,694550.0,Never,Never,Never,...,7,7.0,1.000000,6,6.0,1.0,3,3.0,1.0,1.0000
14582,14583,PH8,1247.0,20.0,2.0,1247.0,694550.0,Never,Never,Never,...,7,9.0,1.285714,6,6.0,1.0,3,3.0,1.0,1.1250
14583,14584,PH8,1247.0,23.0,2.0,1247.0,694550.0,Never,Never,Never,...,7,8.0,1.142857,6,6.0,1.0,3,3.0,1.0,1.0625
14584,14585,PH8,1247.0,26.0,7.0,1247.0,694550.0,Never,Never,Never,...,7,7.0,1.000000,6,6.0,1.0,3,3.0,1.0,1.0000


In [ ]:
df.to_csv('final_with_name.csv', index=False)